In [1]:
from lib.datasets import ATPBind3D

from torchdrug import transforms
from torchdrug import data, core, layers, tasks, metrics, utils, models
from torchdrug.layers import functional
from torchdrug.core import Registry as R

import torch
from torch.utils import data as torch_data
from torch.nn import functional as F
from lib.tasks import NodePropertyPrediction


In [2]:
truncuate_transform = transforms.TruncateProtein(max_length=350, random=False)
protein_view_transform = transforms.ProteinView(view='residue')
transform = transforms.Compose([truncuate_transform, protein_view_transform])

dataset = ATPBind3D(transform=transform)

train_set, valid_set, test_set = dataset.split()
print("train samples: %d, valid samples: %d, test samples: %d" %
      (len(train_set), len(valid_set), len(test_set)))


Split num:  [337, 41, 41]
train samples: 337, valid samples: 41, test samples: 41


In [3]:
from torchdrug import tasks, core, layers
from torchdrug.layers import geometry
graph_construction_model = layers.GraphConstruction(node_layers=[geometry.AlphaCarbonNode()],
                                                    edge_layers=[geometry.SpatialEdge(radius=10.0, min_distance=5),
                                                                 geometry.KNNEdge(
                                                                     k=10, min_distance=5),
                                                                 geometry.SequentialEdge(max_distance=2)],
                                                    edge_feature="residue_type")

zipped_protein = data.Protein.pack([train_set[0]['graph']])
constructed_protein = graph_construction_model(zipped_protein)
print(zipped_protein)
print(graph_construction_model(zipped_protein))
print(zipped_protein.node_feature.type())
zipped_protein = data.Protein.pack([train_set[0]['graph']])


PackedProtein(batch_size=1, num_atoms=[1400], num_bonds=[2798], num_residues=[350])
PackedProtein(batch_size=1, num_atoms=[350], num_bonds=[6029], num_residues=[350])
torch.sparse.LongTensor


In [6]:
from torchdrug import tasks, core, layers
from torchdrug.layers import geometry
import torch
from torch.nn import functional
from lib.tasks import NodePropertyPrediction

graph_construction_model = layers.GraphConstruction(node_layers=[geometry.AlphaCarbonNode()],
                                                    edge_layers=[geometry.SpatialEdge(radius=10.0, min_distance=5),
                                                                 geometry.KNNEdge(
                                                                     k=10, min_distance=5),
                                                                 geometry.SequentialEdge(max_distance=2)],
                                                    edge_feature="residue_type")
task = NodePropertyPrediction(
    models.GearNet(input_dim=21, hidden_dims=[512, 512, 512], num_relation=7,
                   batch_norm=True, concat_hidden=True, short_cut=True, readout="sum").cuda(),
    graph_construction_model=graph_construction_model,
    normalization=False,
    num_mlp_layer=2,
    metric=("micro_auroc")
)

optimizer = torch.optim.Adam(task.parameters(), lr=1e-4)
solver = core.Engine(task, train_set, valid_set, test_set, optimizer,
                     gpus=[0], batch_size=1)
solver.train(num_epoch=10)
solver.evaluate("valid")


21:53:03   Preprocess training set
21:53:04   {'batch_size': 1,
 'class': 'core.Engine',
 'gpus': [0],
 'gradient_interval': 1,
 'log_interval': 100,
 'logger': 'logging',
 'num_worker': 0,
 'optimizer': {'amsgrad': False,
               'betas': (0.9, 0.999),
               'class': 'optim.Adam',
               'eps': 1e-08,
               'lr': 0.0001,
               'weight_decay': 0},
 'scheduler': None,
 'task': {'class': 'NodePropertyPrediction',
          'criterion': 'bce',
          'graph_construction_model': {'class': 'layers.GraphConstruction',
                                       'edge_feature': 'residue_type',
                                       'edge_layers': [SpatialEdge(),
                                                       KNNEdge(),
                                                       SequentialEdge()],
                                       'node_layers': [AlphaCarbonNode()]},
          'metric': 'micro_auroc',
          'model': {'activation': 'relu',
   

{'micro_auroc': tensor(0.7937, device='cuda:0')}